Moodle XML Question Generator, (C) 2019-2020 Kosaku Nagasaka (Kobe University)

# how to use moodle_xqg

The basic usage of moodle_xqg (Moodle XML Question Generator for Python) is as follows.

## loading the package

**moodle_xqg.core** is required mandatory. **moodle_xqgb.qbank.common** is not required but recommended since this includes some useful functions (e.g. deleting duplicate answers).

In [1]:
import moodle_xqg.core as mxqg
import moodle_xqg.qbank.common as mxqg_common

Mostly the following modules are necessary. For example, random integer generations and manipulating mathematical expressions are very common tasks.

In [2]:
import random
import sympy

## minimum example of question generator

Your question generator must be defined as a child class of **Question**, and the class implements the following 5 methods: **question_generate**, **correct_answers_generate**, **incorrect_answers_generate**, **question_text** and **answer_text**.

Your question generator class has to generate the identifier of each quiz and store it in the variable **Quiz.quiz_identifier**. The package distinguish quizzes by this value. Therefore, the package generates only one quiz if you forget to implement this part.

In [3]:
class integer_addition_simple(mxqg.Question):
    def question_generate(self, _quiz_number=0):
        quiz = mxqg.Quiz(name='addition of integers', quiz_number=_quiz_number)
        # generates a quiz data 
        quiz.data = [random.choice([-9,-8,-7,-6,-5,-4,-3,-2,-1,1,2,3,4,5,6,7,8,9]) for i in range(2)]
        quiz.quiz_identifier = hash('{}+{}'.format(quiz.data[0], quiz.data[1]))
        # generates the correct answer
        quiz.data.append(quiz.data[0] + quiz.data[1])
        # generates the correct choice
        ans = { 'fraction': 100, 'data': quiz.data[2] }
        quiz.answers.append(ans)
        return quiz        
    def correct_answers_generate(self, quiz, size=1):
        # nothing to do since we already have the correct answer generated
        pass
    def incorrect_answers_generate(self, quiz, size=4):
        # generates incorrect choices randomly (simple random generation is not recommended though)
        answers = []
        ans = { 'fraction': 0, 'data': quiz.data[2] }
        while len(answers) < size:
            ans['data'] += random.randint(-9,9)
            ans['feedback'] = 'think and compute carefully. it is just a simple addition.'
            if ans['data'] != quiz.data[2]:
                answers.append(dict(ans)) # you may need to copy the object if reuse it
            answers = mxqg_common.answer_union(answers)
        return answers    
    def question_text(self, quiz):
        return 'Choose the result of the following arithmetic.<br />' + \
                mxqg_common.sympy_str_to_text('{}+{}'.format(quiz.data[0], quiz.data[1]))
    def answer_text(self, ans):
        return mxqg_common.sympy_expr_to_text(ans['data'])

To generate actual quizzes, the **generate** method is used. However, before calling the method, we have to generate an instance of the question generator. The number of quizzes to be generated is specified by **size=100** for *generate* (in the default, 10 quizzes will be generated).

In [4]:
ias = integer_addition_simple()
ias_quizzes = mxqg.generate(ias, category='minimum example', size=5)

The result of question generation is an instance of the **Quizzes** class. We can use the methods of the class as follows.

In [5]:
ias_quizzes.listview()

In the following preview, the scores and feedbacks will be appeared as mouse over text.

In [6]:
ias_quizzes.preview(size=2)

Without modifying the definition of the question generator class, we can translate the messages by specifying the *translate* option (dict). However, please note that this does not change the instance, and it changed just a preview only.

In [9]:
ias_quizzes.preview(size=2, translate={'arithmetic':'mathematical expression'})

Use the *show_quiz_number_adminonly* option to disable the quiz id appearance.

In [10]:
ias_quizzes.preview(size=2, show_quiz_number_adminonly=True)

Finally, the **save** method writes the quizzes in the specified file in the Moodle XML format that can be imported to Moodle.

In [7]:
ias_quizzes.save('integer_addition_simple_in_english.xml', translate={'arithmetic':'mathematical expression'})

Moreover, if you do not need the quiz id in the question text, specify **show_quiz_number=False** as an additional option to *save*.

## refined minimum example of question generator

The following generator is a refined one based on the previous minimum example.

In [4]:
class integer_addition(integer_addition_simple):
    def __init__(self, lower=-10, upper=10):
        # now we can specify the range of integers to be generated.
        self.lower = lower
        self.upper = upper
    def question_generate(self, _quiz_number=0):
        quiz = mxqg.Quiz(name='addition of integers', quiz_number=_quiz_number)
        # generates a quiz data (exclude 0 since SymPy can not convert 0 to LaTeX)
        quiz.data = [random.choice(list(set(range(self.lower, self.upper + 1))-{0})) for i in range(2)]
        quiz.quiz_identifier = hash('{}+{}'.format(quiz.data[0], quiz.data[1]))
        # generates the correct answer
        quiz.data.append(quiz.data[0] + quiz.data[1])  
        # generates the correct choice
        ans = { 'fraction': 100, 'data': quiz.data[2] }
        quiz.answers.append(ans)
        return quiz        
    def incorrect_answers_generate(self, quiz, size=7):
        answers = []
        ans = { 'fraction': 0 }
        # using incorrect binary operator
        ans['data'] = quiz.data[0] - quiz.data[1]
        ans['feedback'] = 'The given arithmetic is not a substraction.'
        if ans['data'] != quiz.data[2]:
            answers.append(dict(ans))
        ans['data'] = quiz.data[0] * quiz.data[1]
        ans['feedback'] = 'The given arithmetic is not a multiplication.'
        if ans['data'] != quiz.data[2]:
            answers.append(dict(ans))
        if quiz.data[1] != 0:
            ans['data'] = sympy.Rational(quiz.data[0], quiz.data[1])
            ans['feedback'] = 'The given arithmetic is not a division.'
            if ans['data'] != quiz.data[2]:
                answers.append(dict(ans))
        answers = mxqg_common.answer_union(answers)
        if len(answers) >= size:
            return random.sample(answers,k=size)
        # generates incorrect choices randomly (simple random generation is not recommended though)
        count = 0
        ans['data'] = quiz.data[0] + quiz.data[1]
        while len(answers) < size and count < 10:
            count += 1
            ans['data'] += random.randint(self.lower, self.upper)
            ans['feedback'] = 'think and compute carefully. it is just a simple addition.'
            if ans['data'] != quiz.data[2]:
                answers.append(dict(ans))
            answers = mxqg_common.answer_union(answers)
        return answers    

Now we can control the behaviors of the generator by its initial arguments.

In [5]:
ia = integer_addition(lower=-500, upper=500)
ia_quizzes = mxqg.generate(ia, category='refined minimum example')

Please take a look at the mouse over text.

In [7]:
ia_quizzes.preview(size=2)

With the **internal=True** option, you can save the quizzes in the pickle format other than the Moodle XML format.

In [15]:
ia_quizzes.save('integer_addition_in_english.pickle', internal=True)

To load the exported quizzes, use the **load_internal** method.

In [16]:
ia_quizzes = mxqg.load_internal('integer_addition_in_english.pickle')

In [17]:
ia_quizzes.preview(size=1)

Moreover, you can load the quizzes in the Moodle XML format though some internal information (internal quiz *data*, quiz id *quiz_number* and *quiz_identifier*) will not be recovered. For this purpose, use the **load_xml** method as follows.

In [18]:
ia_quizzes.save('integer_addition_in_english.xml')
ia_quizzes = mxqg.load_xml('integer_addition_in_english.xml')

For example, you have to specify the option **show_quiz_number=False** when you use the methods: *preview* and *save* since already the quiz id is included in the question text as text. Otherwise, the quiz id will be duplicated.

In [19]:
ia_quizzes.preview(size=1, show_quiz_number=False)